## Engenharia de características

####  Agregando múlpitlos fatores e agrupando mêses

Esse código é rodado por 015

####  Agregando Múlpitlos

> Esse é o momento em que a previsão de importação de um produto deixa de ser feita somente com base em seu próprio histórico de importação, passando a ser feita também considerando outras interações, que nesses caso será com a importação de outros produtos fertilizantes, passando a ser denominada uma previsão multifatorial. O objetivo da inteligencia artificial é nos permitir fazer esse processo com relativamente pouco serviço de codificação.
>
> Por exemplo: A chegada ou o volume de chagda de um determinado produto pode dar indicativo de quando e quanto de um outro produto chegará no momento seguinte. Isso daqui para frente será considerado. 
>
> Testes feitos pelo autor, que serão apresentados, mostram que, com a previsão multifatorial, a [precisão](https://pt.wikipedia.org/wiki/Precis%C3%A3o#:~:text=Em%20engenharia%2C%20ci%C3%AAncia%2C%20ind%C3%BAstria%20e,de%20repeti%C3%A7%C3%B5es%20da%20mesma%20an%C3%A1lise.) medida aumentou de 70 para mais de 95%.

In [18]:
# importação das bibliotecas a serem utilizadas
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [19]:
i=pd.read_csv('~/comex/data/imp4.csv').groupby(by=['ANO','MES','NCM'])['KG'].sum().reset_index()\
.sort_values(by=['ANO','MES','NCM']).reset_index(drop=True)
l=i.NCM.unique().tolist()
l

['31022100-0',
 '31021010-0',
 '31042090-0',
 '31054000-0',
 '31059090-1',
 '31055900-1']

Quando a previsão de um produto for ser executada, o algorítmo ter conhecimento de todo o histórico desse produto, né claro!

Porém, além disso, o algoritmo terá conhecimento do histórico dos demais produtos também!

Para que no momento da previsão de um produto a importaçãos dos outros produtos possam ser inseridas se serem elas mesmas previsões, o que causaria previsão em cima de previsão, que não é desejável,aqui isso é evitado usando o input dos outros fertilizantes com 1 mês de atraso.

In [20]:
i_ad=i.copy()

i_ad.MES=i_ad.MES+1

i_ad.loc[i_ad.MES==13,'ANO']=i_ad.loc[i_ad.MES==13]['ANO']+1
i_ad.loc[i_ad.MES==13,'MES']=1
i_ad=i_ad.sort_values(by=['ANO','MES','NCM'])

i1=i_ad[i_ad.NCM==l[0]][['ANO','MES','KG']].rename(columns={'KG':'k1'})
i2=i_ad[i_ad.NCM==l[1]][['ANO','MES','KG']].rename(columns={'KG':'k2'})
i3=i_ad[i_ad.NCM==l[2]][['ANO','MES','KG']].rename(columns={'KG':'k3'})
i4=i_ad[i_ad.NCM==l[3]][['ANO','MES','KG']].rename(columns={'KG':'k4'})
i5=i_ad[i_ad.NCM==l[4]][['ANO','MES','KG']].rename(columns={'KG':'k5'})
i6=i_ad[i_ad.NCM==l[5]][['ANO','MES','KG']].rename(columns={'KG':'k6'})
i_agg=i1.merge(i2,how='outer').merge(i3,how='outer').merge(i4,how='outer').merge(i5,how='outer').merge(i6,how='outer')
i_agg=i_agg.sort_values(by=['ANO','MES'])
i_agg=i_agg.replace({np.nan:0})
i_agg.to_csv('~/comex/data/i_agg.csv',index=False)

i_agg[['ANO','MES']]=i_agg[['ANO','MES']].astype('category')
i_agg_full=i_agg.groupby(by=['ANO','MES'])[['k1','k2','k3','k4','k5','k6']].sum().reset_index()


i[['ANO','MES']]=i[['ANO','MES']].astype('category')
i_full=i.groupby(by=['ANO','MES'])[['KG']].sum().reset_index()

if agg==1:
    i=i.merge(i_agg_full)

In [21]:
i_agg

,ANO,MES,k1,k2,k3,k4,k5,k6
0,1997,4,6004584.0,0.0,0.0,0.0,0.0,0.0
1,1997,5,11049492.0,531376.0,7045252.0,3515344.0,0.0,0.0
2,1997,6,4202529.0,0.0,4698701.0,2038700.0,0.0,0.0
290,1997,7,0.0,0.0,0.0,9449094.0,0.0,0.0
216,1997,8,0.0,0.0,5549943.0,8711405.0,0.0,0.0
...,...,...,...,...,...,...,...,...
177,2023,7,60699616.0,37231377.0,58835431.0,36008475.0,1130800.0,8405699.0
178,2023,8,26420845.0,3101138.0,91427020.0,26499722.0,3028800.0,19320677.0
179,2023,9,4000000.0,10500000.0,37734795.0,16676671.0,0.0,0.0
180,2023,10,3024113.0,51600979.0,73775000.0,23781051.0,0.0,0.0


#### Agrupa Meses

> Nesse momento a planilha recebe seus próprios valores, que originalmente são agrupados em mês, mas que de agora em diante terão também valores agrupados por bimestre e trimestre tbm.
>
> A tabela ganha uma nova coluna, denominada 'gruppo_mes' que servirá para identificar se a importação declarada é referente ao mês, bimestre ou trimestre.

> Note que os bimestres e trimestres não são necessariamente começados no mês 01,  e sim começados sempre no mês a ser predito. Na visão do autor, essa medida proporciona o máximo uso da informação no momento da previsão.

In [9]:
um=i.MES.iloc[-1]
um

10

In [10]:
i[['ANO','MES']]=i[['ANO','MES']].astype(int)

# Mês

In [11]:
im=i.copy()


d={1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,11:11,12:12}

im.MES=im.MES.replace(d)

im=im.groupby(by=['ANO','MES','NCM'])[im.select_dtypes(include='float64').columns.tolist()].sum().reset_index()
im

,ANO,MES,NCM,KG,k1,k2,k3,k4,k5,k6
0,1997,3,31022100-0,6004584.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1997,4,31021010-0,531376.0,6004584.0,0.0,0.0,0.0,0.0,0.0
2,1997,4,31022100-0,11049492.0,6004584.0,0.0,0.0,0.0,0.0,0.0
3,1997,4,31042090-0,7045252.0,6004584.0,0.0,0.0,0.0,0.0,0.0
4,1997,4,31054000-0,3515344.0,6004584.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
963,2023,10,31022100-0,114750000.0,3024113.0,51600979.0,73775000.0,23781051.0,0.0,0.0
964,2023,10,31042090-0,37437366.0,3024113.0,51600979.0,73775000.0,23781051.0,0.0,0.0
965,2023,10,31054000-0,14695000.0,3024113.0,51600979.0,73775000.0,23781051.0,0.0,0.0
966,2023,10,31055900-1,9516500.0,3024113.0,51600979.0,73775000.0,23781051.0,0.0,0.0


# Bimestre

In [12]:
ib=i.copy()

if um%2==0:
    d={1:1,2:1,3:3,4:3,5:5,6:5,7:7,8:7,9:9,10:9,11:11,12:11}
    
if (um+1)%2==0:
    d={1:12,2:2,3:2,4:4,5:4,6:6,7:6,8:8,9:8,10:10,11:10,12:12}
    ib.loc[ib.MES.isin([1]),'ANO']=ib[ib.MES.isin([1])]['ANO']-1

ib.MES=ib.MES.replace(d)

ib=ib.groupby(by=['ANO','MES','NCM'])[ib.select_dtypes(include='float64').columns.tolist()].sum().reset_index()
ib

,ANO,MES,NCM,KG,k1,k2,k3,k4,k5,k6
0,1997,3,31021010-0,531376.0,6004584.0,0.0,0.0,0.0,0.0,0.0
1,1997,3,31022100-0,17054076.0,6004584.0,0.0,0.0,0.0,0.0,0.0
2,1997,3,31042090-0,7045252.0,6004584.0,0.0,0.0,0.0,0.0,0.0
3,1997,3,31054000-0,3515344.0,6004584.0,0.0,0.0,0.0,0.0,0.0
4,1997,5,31022100-0,4202529.0,11049492.0,531376.0,7045252.0,3515344.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
614,2023,9,31022100-0,117774113.0,7024113.0,62100979.0,111509795.0,40457722.0,0.0,0.0
615,2023,9,31042090-0,111212366.0,7024113.0,62100979.0,111509795.0,40457722.0,0.0,0.0
616,2023,9,31054000-0,38476051.0,7024113.0,62100979.0,111509795.0,40457722.0,0.0,0.0
617,2023,9,31055900-1,9516500.0,3024113.0,51600979.0,73775000.0,23781051.0,0.0,0.0


# Trimestre

In [13]:
it=i.copy()

if um%3==0:
    d={1:1,2:1,3:1,4:4,5:4,6:4,7:7,8:7,9:7,10:10,11:10,12:10}

if (um+1)%3==0:
    d={1:12,2:12,3:3,4:3,5:3,6:6,7:6,8:6,9:9,10:9,11:9,12:12}
    it.loc[it.MES.isin([1,2]),'ANO']=it[it.MES.isin([1,2])]['ANO']-1


if (um+2)%3==0:
    d={1:11,2:2,3:2,4:2,5:5,6:5,7:5,8:8,9:8,10:8,11:11,12:11}
    it.loc[it.MES==1,'ANO']=it[it.MES==1]['ANO']-1

it.MES=it.MES.replace(d)

it=it.groupby(by=['ANO','MES','NCM'])[it.select_dtypes(include='float64').columns.tolist()].sum().reset_index()
it

,ANO,MES,NCM,KG,k1,k2,k3,k4,k5,k6
0,1997,2,31021010-0,531376.0,6004584.0,0.0,0.0,0.0,0.0,0.0
1,1997,2,31022100-0,17054076.0,6004584.0,0.0,0.0,0.0,0.0,0.0
2,1997,2,31042090-0,7045252.0,6004584.0,0.0,0.0,0.0,0.0,0.0
3,1997,2,31054000-0,3515344.0,6004584.0,0.0,0.0,0.0,0.0,0.0
4,1997,5,31022100-0,4202529.0,11049492.0,531376.0,7045252.0,3515344.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
463,2023,8,31022100-0,121774113.0,33444958.0,65202117.0,202936815.0,66957444.0,3028800.0,19320677.0
464,2023,8,31042090-0,148947161.0,33444958.0,65202117.0,202936815.0,66957444.0,3028800.0,19320677.0
465,2023,8,31054000-0,55152722.0,33444958.0,65202117.0,202936815.0,66957444.0,3028800.0,19320677.0
466,2023,8,31055900-1,9516500.0,3024113.0,51600979.0,73775000.0,23781051.0,0.0,0.0


In [14]:
i=pd.concat([im.assign(grupo_mes='1'),\
           ib.assign(grupo_mes='2'),\
           it.assign(grupo_mes='3')]\
           )

In [15]:
i.ANO=i.ANO.astype(int)
i=i.sort_values(by=['ANO','MES','NCM','grupo_mes'])

In [16]:
i.to_csv('~/comex/data/imp5.csv',index=False)
i.NCM.unique()

array(['31021010-0', '31022100-0', '31042090-0', '31054000-0',
       '31059090-1', '31055900-1'], dtype=object)